<a href="https://colab.research.google.com/github/romip111/hackathongrupo11/blob/main/SmartCityV1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import warnings

# Ignorar advertencias futuras (comunes con la codificación)
warnings.filterwarnings('ignore', category=FutureWarning)

# --- 1. Carga y Preparación de Datos ---

# Cargar el archivo CSV
try:
    df = pd.read_csv('accidentes_final_5_columnas_limpio.csv')
except FileNotFoundError:
    print("Error: No se encontró el archivo 'accidentes_final_5_columnas_limpio.csv'.")
    exit()

# Definir las características (X) y el objetivo (y)
features = ['Calle_Uno', 'Calle_Dos', 'Comuna', 'Tipo_Accid']
target = 'Frecuencia'

X = df[features]
y = df[target]

# --- 2. Limpieza de Datos ---
# Reemplazar marcadores de "no aplica" y rellenar valores nulos (NaN)
# Esto asegura que el codificador los trate como una categoría más.
placeholder = 'Desconocido'
X = X.replace('s/interseccion', placeholder)
X = X.fillna(placeholder)

print(f"Datos cargados: {df.shape[0]} registros.")
print("-----------------------------------")


# --- 3. Codificación de Variables (El Paso Clave) ---

# a) Codificar las características (X)
# Usamos OrdinalEncoder porque Random Forest puede manejar
# números enteros categóricos. Es eficiente para muchas categorías (como calles).
# 'handle_unknown' permite al modelo manejar calles o comunas que no vio en el entrenamiento.
ordinal_encoder = OrdinalEncoder(
    handle_unknown='use_encoded_value',
    unknown_value=-1 # Asigna -1 a cualquier categoría desconocida
)
X_encoded = ordinal_encoder.fit_transform(X)

# b) Codificar la variable objetivo (y)
# Usamos LabelEncoder para la columna 'Frecuencia'
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Guardar las clases de frecuencia (ej: 'muy frecuente', 'común', etc.)
frecuencia_classes = label_encoder.classes_
print(f"Categorías de frecuencia encontradas: {list(frecuencia_classes)}")
print("-----------------------------------")


# --- 4. División de Datos ---
# Dividir en 80% entrenamiento y 20% prueba
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded,
    y_encoded,
    test_size=0.2,
    random_state=42,
    stratify=y_encoded # Asegura proporción de frecuencias en ambos sets
)


# --- 5. Entrenamiento del Modelo ---
print("Entrenando el modelo Random Forest...")
model = RandomForestClassifier(
    n_estimators=100,  # Número de "árboles" en el bosque
    random_state=42,   # Para resultados reproducibles
    n_jobs=-1          # Usar todos los procesadores disponibles
)
model.fit(X_train, y_train)
print("¡Modelo entrenado!")
print("-----------------------------------")


# --- 6. Evaluación del Modelo ---
print("Evaluando el modelo con los datos de prueba...")
y_pred = model.predict(X_test)

# Calcular la precisión
accuracy = accuracy_score(y_test, y_pred)
print(f"Precisión (Accuracy) del modelo: {accuracy:.2%}")
print("\nReporte de Clasificación:")
# Mostrar el reporte con los nombres originales de las frecuencias
print(classification_report(y_test, y_pred, target_names=frecuencia_classes))
print("-----------------------------------")


# --- 7. Cómo Realizar Nuevas Predicciones ---
print("Ejemplo de predicción para nuevos datos:")

# Crear datos de ejemplo (pueden ser 1 o varias filas)
# Usamos datos que sabemos están en el CSV para un buen ejemplo
nuevos_datos = pd.DataFrame({
    'Calle_Uno': ['2 ORIENTE', 'CALLE LARGA', 'UNA CALLE NUEVA'],
    'Calle_Dos': ['8 NORTE', 'LA PAMPILLA', 'OTRA CALLE'],
    'Comuna': ['VIÑA DEL MAR', 'CALLE LARGA', 'QUINTERO'],
    'Tipo_Accid': ['ATROPELLO', 'COLISION POR ALCANCE', 'CHOQUE']
})

# IMPORTANTE: Aplicar la misma limpieza que a los datos de entrenamiento
nuevos_datos = nuevos_datos.replace('s/interseccion', placeholder)
nuevos_datos = nuevos_datos.fillna(placeholder)

# 1. Transformar los nuevos datos con el MISMO codificador (ordinal_encoder)
nuevos_datos_encoded = ordinal_encoder.transform(nuevos_datos)
print(f"\nDatos de entrada (codificados):\n {nuevos_datos_encoded}")

# 2. Predecir con el modelo entrenado
predicciones_encoded = model.predict(nuevos_datos_encoded)

# 3. "Decodificar" las predicciones para obtener el texto
predicciones_labels = label_encoder.inverse_transform(predicciones_encoded)

# 4. Mostrar resultados
print("\nResultados de la predicción:")
for i in range(len(nuevos_datos)):
    print(f"  -> Caso {i+1} ({nuevos_datos.iloc[i].values}): {predicciones_labels[i]}")

Datos cargados: 18315 registros.
-----------------------------------
Categorías de frecuencia encontradas: ['común', 'esporádico', 'muy frecuente']
-----------------------------------
Entrenando el modelo Random Forest...
¡Modelo entrenado!
-----------------------------------
Evaluando el modelo con los datos de prueba...
Precisión (Accuracy) del modelo: 97.11%

Reporte de Clasificación:
               precision    recall  f1-score   support

        común       0.88      0.81      0.84       252
   esporádico       0.84      0.38      0.52        84
muy frecuente       0.98      1.00      0.99      3327

     accuracy                           0.97      3663
    macro avg       0.90      0.73      0.79      3663
 weighted avg       0.97      0.97      0.97      3663

-----------------------------------
Ejemplo de predicción para nuevos datos:

Datos de entrada (codificados):
 [[ 3.00e+01  8.20e+01  4.50e+01  0.00e+00]
 [ 4.09e+02  1.32e+03  5.00e+00  1.90e+01]
 [-1.00e+00 -1.00e+00  3

In [ ]:
import joblib

# --- 8. Guardar el Entrenamiento para el Futuro ---
print("Guardando el modelo y los codificadores...")

# Crear un diccionario con los 3 objetos que necesitamos
entrenamiento_completo = {
    'modelo': model,
    'codificador_ordinal': ordinal_encoder,
    'codificador_labels': label_encoder,
    'placeholder_limpieza': placeholder # Guardamos también el valor de 'Desconocido'
}

# Guardar el diccionario en un solo archivo
joblib.dump(entrenamiento_completo, 'entrenamiento_accidentes_rf.joblib')

print(f"¡Entrenamiento guardado con éxito en 'entrenamiento_accidentes_rf.joblib'!")

Guardando el modelo y los codificadores...
¡Entrenamiento guardado con éxito en 'entrenamiento_accidentes_rf.joblib'!


In [ ]:
import joblib
import pandas as pd

# --- 1. Cargar el Entrenamiento Guardado ---

try:
    # Cargar el diccionario completo desde el archivo
    entrenamiento = joblib.load('entrenamiento_accidentes_rf.joblib')
except FileNotFoundError:
    print("Error: No se encontró el archivo 'entrenamiento_accidentes_rf.joblib'.")
    print("Asegúrate de ejecutar primero el script de entrenamiento.")
    exit()


# Extraer los objetos del diccionario
model = entrenamiento['modelo']
ordinal_encoder = entrenamiento['codificador_ordinal']
label_encoder = entrenamiento['codificador_labels']
placeholder = entrenamiento['placeholder_limpieza']

print("Modelo y codificadores cargados con éxito.")
print("-----------------------------------")


# --- 2. Preparar los Nuevos Datos ---
# (Estos son los datos que quieres predecir)

nuevos_datos = pd.DataFrame({
    'Calle_Uno': ['2 ORIENTE', 'UNA CALLE NUEVA', 'O\'HIGGINS'],
    'Calle_Dos': ['8 NORTE', 'OTRA CALLE', 's/interseccion'], # 's/interseccion' será limpiado
    'Comuna': ['VIÑA DEL MAR', 'QUINTERO', 'SAN ESTEBAN'],
    'Tipo_Accid': ['ATROPELLO', 'CHOQUE', 'COLISION POR ALANCE']
})

print("Datos a predecir:")
print(nuevos_datos)


# --- 3. Proceso de Predicción ---

# PASO A: Aplicar la MISMA limpieza
# Es crucial reemplazar 's/interseccion' y nulos con el mismo placeholder
nuevos_datos_limpios = nuevos_datos.replace('s/interseccion', placeholder)
nuevos_datos_limpios = nuevos_datos_limpios.fillna(placeholder)

# PASO B: Codificar los datos con el codificador CARGADO
datos_codificados = ordinal_encoder.transform(nuevos_datos_limpios)
print(f"\nDatos codificados (listos para el modelo):\n {datos_codificados}")

# PASO C: Predecir con el modelo CARGADO
prediccion_numerica = model.predict(datos_codificados)

# PASO D: Decodificar el resultado con el codificador CARGADO
prediccion_texto = label_encoder.inverse_transform(prediccion_numerica)


# --- 4. Mostrar Resultados ---
print("\n--- Resultados de la Predicción ---")

for i in range(len(nuevos_datos)):
    fila = nuevos_datos.iloc[i]
    print(f"  -> ({fila['Calle_Uno']}, {fila['Comuna']}, {fila['Tipo_Accid']}): {prediccion_texto[i]}")

Modelo y codificadores cargados con éxito.
-----------------------------------
Datos a predecir:
         Calle_Uno       Calle_Dos        Comuna           Tipo_Accid
0        2 ORIENTE         8 NORTE  VIÑA DEL MAR            ATROPELLO
1  UNA CALLE NUEVA      OTRA CALLE      QUINTERO               CHOQUE
2        O'HIGGINS  s/interseccion   SAN ESTEBAN  COLISION POR ALANCE

Datos codificados (listos para el modelo):
 [[ 3.000e+01  8.200e+01  4.500e+01  0.000e+00]
 [-1.000e+00 -1.000e+00  3.200e+01  2.000e+00]
 [ 1.843e+03  7.590e+02  3.500e+01 -1.000e+00]]

--- Resultados de la Predicción ---
  -> (2 ORIENTE, VIÑA DEL MAR, ATROPELLO): muy frecuente
  -> (UNA CALLE NUEVA, QUINTERO, CHOQUE): muy frecuente
  -> (O'HIGGINS, SAN ESTEBAN, COLISION POR ALANCE): muy frecuente


Overfitting?

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import warnings

# Ignorar advertencias futuras (comunes con la codificación)
warnings.filterwarnings('ignore', category=FutureWarning)

# --- 1. Carga y Preparación de Datos ---

# Cargar el archivo CSV
try:
    df = pd.read_csv('accidentes_final_5_columnas_limpio.csv')
except FileNotFoundError:
    print("Error: No se encontró el archivo 'accidentes_final_5_columnas_limpio.csv'.")
    exit()

# Definir las características (X) y el objetivo (y)
features = ['Calle_Uno', 'Calle_Dos', 'Comuna', 'Tipo_Accid']
target = 'Frecuencia'

X = df[features]
y = df[target]

# --- 2. Limpieza de Datos ---
# Reemplazar marcadores de "no aplica" y rellenar valores nulos (NaN)
placeholder = 'Desconocido'
X = X.replace('s/interseccion', placeholder)
X = X.fillna(placeholder)

print(f"Datos cargados: {df.shape[0]} registros.")
print("-----------------------------------")


# --- 3. Codificación de Variables (El Paso Clave) ---

# a) Codificar las características (X)
ordinal_encoder = OrdinalEncoder(
    handle_unknown='use_encoded_value',
    unknown_value=-1 # Asigna -1 a cualquier categoría desconocida
)
X_encoded = ordinal_encoder.fit_transform(X)

# b) Codificar la variable objetivo (y)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Guardar las clases de frecuencia (ej: 'muy frecuente', 'común', etc.)
frecuencia_classes = label_encoder.classes_
print(f"Categorías de frecuencia encontradas: {list(frecuencia_classes)}")
print("-----------------------------------")


# --- 4. División de Datos ---
# Dividir en 80% entrenamiento y 20% prueba
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded,
    y_encoded,
    test_size=0.2,
    random_state=42,
    stratify=y_encoded # Asegura proporción de frecuencias en ambos sets
)


# --- 5. Entrenamiento del Modelo ---
print("Entrenando el modelo Random Forest...")
model = RandomForestClassifier(
    n_estimators=100,  # Número de "árboles" en el bosque
    random_state=42,   # Para resultados reproducibles
    n_jobs=-1          # Usar todos los procesadores disponibles
)
model.fit(X_train, y_train)
print("¡Modelo entrenado!")
print("-----------------------------------")


# --- 6. Evaluación del Modelo (Verificación de Overfitting) ---
print("Evaluando el modelo...")

# 6a. Evaluación en el conjunto de PRUEBA (datos no vistos)
y_test_pred = model.predict(X_test)
accuracy_test = accuracy_score(y_test, y_test_pred)
print(f"Precisión (Accuracy) en PRUEBA: {accuracy_test:.2%}")

# 6b. Evaluación en el conjunto de ENTRENAMIENTO (datos vistos)
y_train_pred = model.predict(X_train)
accuracy_train = accuracy_score(y_train, y_train_pred)
print(f"Precisión (Accuracy) en ENTRENAMIENTO: {accuracy_train:.2%}")


print("\n--- Conclusión sobre Overfitting ---")
# Una diferencia grande entre entrenamiento y prueba indica Overfitting
diferencia = accuracy_train - accuracy_test
if diferencia > 0.15:
    print(f"🚨 ALTO RIESGO DE OVERFITTING ({diferencia:.2%}): La precisión de entrenamiento es significativamente mayor. El modelo podría estar memorizando.")
elif diferencia > 0.05:
    print(f"⚠️ RIESGO MODERADO DE OVERFITTING ({diferencia:.2%}): Hay una brecha notable. Considere regularizar el modelo.")
else:
    print("✅ RIESGO BAJO DE OVERFITTING: Las precisiones son cercanas. El modelo parece generalizar bien.")
print("-----------------------------------")


print("\nReporte de Clasificación (Conjunto de Prueba):")
# Mostrar el reporte con los nombres originales de las frecuencias
print(classification_report(y_test, y_test_pred, target_names=frecuencia_classes))
print("-----------------------------------")


# --- 7. Cómo Realizar Nuevas Predicciones ---
print("Ejemplo de predicción para nuevos datos:")

# Crear datos de ejemplo (pueden ser 1 o varias filas)
nuevos_datos = pd.DataFrame({
    'Calle_Uno': ['2 ORIENTE', 'CALLE LARGA', 'UNA CALLE NUEVA'],
    'Calle_Dos': ['8 NORTE', 'LA PAMPILLA', 'OTRA CALLE'],
    'Comuna': ['VIÑA DEL MAR', 'CALLE LARGA', 'QUINTERO'],
    'Tipo_Accid': ['ATROPELLO', 'COLISION POR ALCANCE', 'CHOQUE']
})

# IMPORTANTE: Aplicar la misma limpieza que a los datos de entrenamiento
nuevos_datos = nuevos_datos.replace('s/interseccion', placeholder)
nuevos_datos = nuevos_datos.fillna(placeholder)

# 1. Transformar los nuevos datos con el MISMO codificador (ordinal_encoder)
nuevos_datos_encoded = ordinal_encoder.transform(nuevos_datos)
print(f"\nDatos de entrada (codificados):\n {nuevos_datos_encoded}")

# 2. Predecir con el modelo entrenado
predicciones_encoded = model.predict(nuevos_datos_encoded)

# 3. "Decodificar" las predicciones para obtener el texto
predicciones_labels = label_encoder.inverse_transform(predicciones_encoded)

# 4. Mostrar resultados
print("\nResultados de la predicción:")
for i in range(len(nuevos_datos)):
    print(f"  -> Caso {i+1} ({nuevos_datos.iloc[i].values}): {predicciones_labels[i]}")

Datos cargados: 18315 registros.
-----------------------------------
Categorías de frecuencia encontradas: ['común', 'esporádico', 'muy frecuente']
-----------------------------------
Entrenando el modelo Random Forest...
¡Modelo entrenado!
-----------------------------------
Evaluando el modelo...
Precisión (Accuracy) en PRUEBA: 97.11%
Precisión (Accuracy) en ENTRENAMIENTO: 100.00%

--- Conclusión sobre Overfitting ---
✅ RIESGO BAJO DE OVERFITTING: Las precisiones son cercanas. El modelo parece generalizar bien.
-----------------------------------

Reporte de Clasificación (Conjunto de Prueba):
               precision    recall  f1-score   support

        común       0.88      0.81      0.84       252
   esporádico       0.84      0.38      0.52        84
muy frecuente       0.98      1.00      0.99      3327

     accuracy                           0.97      3663
    macro avg       0.90      0.73      0.79      3663
 weighted avg       0.97      0.97      0.97      3663

---------